# Tareas de SPSI
> Ana Buendía Ruiz-Azuaga
>
> Paula Villanueva Núñez

## Tarea 1
**Algoritmo extendedido de Euclides**

In [175]:
def bxeuc(m,n):
    '''Función que calcula el máximo común divisor de dos números enteros m y n'''
    
    g = 1
    
    while m%2 == 0 and n%2 == 0 :
        m = m/2
        n = n/2
        g = 2*g
    
    r = m
    s = n
    A = 1
    B = 0
    C = 0
    D = 1
    
    while r != 0:
    
        while r%2 == 0:
            r = r/2
            if A%2 == 0 and B%2 == 0:
                A = A/2
                B = B/2
            else:
                A = (A+n)/2
                B = (B-m)/2

        while s%2 == 0:
            s = s/2
            if C%2 == 0 and D%2 == 0:
                C = C/2
                D = D/2
            else:
                C = (C+n)/2
                D = (D-m)/2

        if r>=s:
            r = r-s
            A = A-C
            B = B-D
        else:
            s = s-r
            C = C-A
            D = D-B

        if r == 0:
            u = C
            v = D
            g = g*s
    
    return g, u, v
        

In [176]:
aux = bxeuc(693,609)
print(aux)

(21.0, -181.0, 206.0)


In [177]:
# not(m & 1) -> Comprueba si el número es par. Devuelve true si es par
def bxeuc(m,n):
    shift = 0
    
    if m == 0:
        return n, 0, 1
    if n == 0:
        return m, 1, 0
    
    # Gestionamos numeros negativos
    signo_m = 1
    signo_n = 1
    
    if m < 0:
        m = -m
        signo_m = -1
    if n < 0:
        n = -n
        signo_n = -1
    
    
    while not(m & 1) and not(n & 1):
        m >>= 1
        n >>= 1
        
        shift = shift+1
        
    # m0 y n0 son m y n quitando el factor comun 2, por ejemplo 4,2 pasa a 2,1
    m0 = m
    n0 = n
    
    sm = 1
    sn = 0
    
    tm = 0
    tn = 1
    
    
    while not(m & 1): # Si m es par
        if not (not(sm & 1) and not(sn & 1) ): # Si sm o sn alguno no es par
            # garantizamos que sm y sn sean pares
            sm = sm - n0
            sn = sn + m0
            
        # Quitamos el factor comun 2 de m, sm y sn
        m >>= 1
        sm >>= 1
        sn >>= 1
    
    while n != 0:
        while not(n & 1): # SI n par
            if not (not(tm & 1) and not(tn & 1)): # Si tm o tn alguno no es par
                # Aseguramos que tm y tn son pares
                tm = tm - n0
                tn = tn + m0
                
            # Eliminamos el factor comun 2 de n, tm y tn
            n >>= 1
            tm >>= 1
            tn >>= 1
        
        # Si m>n intercambiamos las variables
        if m > n:
            t = m
            m = n
            n = t
            
            t = sm
            sm = tm
            tm = t
            
            t = sn
            sn = tn
            tn = t
            
        
        # Vamos quitando tantas veces m a n
        n = n - m
        tm = tm - sm
        tn = tn - sn

    return m << shift, signo_m*sm, signo_n*sn
    
    

In [178]:
bxeuc(16,-4)

(4, 0, -1)

## Tarea 2
**Máquina Enigma**

In [39]:
import sys

In [54]:
ANILLO_BASE = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

In [55]:
class Rotor:
    def __init__(self, notch, ringstellung, wiring):
        '''
        ringstellung es un caracter
        '''
        self.anillo = ANILLO_BASE
        self.notch = notch
        self.ringstellung = ringstellung
        self.wiring = wiring

        
    def aplicar_ringstellung(self, ringstellung):
        '''
        Metodo para aplicar el ringstellung
        '''
        indice_anillo = ANILLO_BASE.find(ringstellung)
        
        self.wiring = list(self.wiring)
        
        for i in range (0, len(self.wiring)):
            indice = ANILLO_BASE.find(self.wiring[i]) + indice_anillo
            self.wiring[i] = ANILLO_BASE[indice % 26]
            
        cadena = ""
        for c in self.wiring:
            cadena = cadena + c
        
        self.wiring = cadena
        self.wiring = self.wiring[-indice_anillo:]+self.wiring[:-indice_anillo]
        
        
        self.ringstellung = ringstellung
        
    
    def aplicar_grundstellung(self, grundstellung):
        '''
        Aplica grundstellung al rotor
        '''
        
        # Estar en la ventanita es estar al principio de la cadena
        indice = self.anillo.find(grundstellung)
        
        self.anillo = self.anillo[indice:] + self.anillo[:indice]
        self.wiring = self.wiring[indice:] + self.wiring[:indice]
        

    def cifrar_caracter(self, car):
        # Guardo posicion de la letra en el alfabeto
        indice = ANILLO_BASE.find(car)
        
        # Accedemos a wiring de la letra
        letra = self.wiring[indice]
        
        # Convertimos la letra segun corresponde en wiring
        ind = self.anillo.find(letra)
        
        return ANILLO_BASE[ind]
    
    def inv_cifrar_caracter(self, car):
        # Guardo posicion de la letra en el alfabeto
        indice = ANILLO_BASE.find(car)
        
        # Accedemos a wiring de la letra
        letra = self.anillo[indice]
        
        # Convertimos la letra segun corresponde en wiring
        ind = self.wiring.find(letra)
        
        return ANILLO_BASE[ind]
    
    def rotar(self):
        self.anillo = self.anillo[1:] + self.anillo[:1]
        self.wiring = self.wiring[1:] + self.wiring[:1]

    
    def comprobar_vuelta_completa(self):
        return self.anillo[0] in self.notch


In [56]:
ROTOR_I = {"notch": "Q", "ringstellung": "A", "wiring": "EKMFLGDQVZNTOWYHXUSPAIBRCJ"}
    
ROTOR_II = {"notch": "E", "ringstellung": "A", "wiring": "AJDKSIRUXBLHWTMCQGZNPYFVOE"}
    
ROTOR_III = {"notch": "V", "ringstellung": "A", "wiring": "BDFHJLCPRTXVZNYEIWGAKMUSQO"}
    
ROTOR_IV = {"notch": "J", "ringstellung": "A", "wiring": "ESOVPZJAYQUIRHXLNFTGKDCMWB"}
    
ROTOR_V = {"notch": "Z", "ringstellung": "A", "wiring": "VZBRGITYUPSDNHLXAWMJQOFECK"}
    
ROTOR_VI = {"notch": "ZM", "ringstellung": "A", "wiring": "JPGVOUMFYQBENHZRDKASXLICTW"}
    
ROTOR_VII = {"notch": "ZM", "ringstellung": "A", "wiring": "NZJHGRCXMYSWBOUFAIVLPEKQDT"}
    
ROTOR_VIII = {"notch": "ZM", "ringstellung": "A", "wiring": "FKQHTLXOCBJSPDZRAMEWNIUYGV"}
    
ROTOR_BETA = {"notch": "", "ringstellung": "A", "wiring": "LEYJVCNIXWPBQMDRTAKZGFUHOS"}
    
ROTOR_GAMMA = {"notch": "", "ringstellung": "A", "wiring": "FSOKANUERHMBTIYCWLQPZXVGJD"}

In [65]:
class Reflector:
    
    def __init__(self, config_reflector):
        self.reflector = list(config_reflector)
        
    def reflejar(self, car):
        indice = ANILLO_BASE.find(car)
        
        # Accedemos a wiring de la letra
        letra = self.reflector[indice % 26]
        
        # Convertimos la letra segun corresponde en wiring
        ind = ANILLO_BASE.find(letra)
        return ANILLO_BASE[ind % 26]

In [66]:
REFLECTOR_B = Reflector("YRUHQSLDPXNGOKMIEBFZCWVJAT")
    
REFLECTOR_C = Reflector("FVPJIAOYEDRZXWGCTKUQSBNMHL")
    
REFLECTOR_B_THIN = Reflector("ENKQAUYWJICOPBLMDXZVFTHRGS")
    
REFLECTOR_C_THIN = Reflector("RDOBJNTKVEHMLFCWZAXGYIPSUQ")

In [67]:
class Plugboard:
    def __init__(self, sustituciones):
        parejas = sustituciones.split(" ")
        
        self.mapa = {}
        
        # Asignamos cada letra consigo misma
        for letra in ANILLO_BASE:
            self.mapa[letra] = letra
        # Intercambiamos las letras indicadas
        for elem in parejas:
            self.mapa[elem[0]] = elem[1]
            self.mapa[elem[1]] = elem[0]
    
    def sustituir_letra(self, car):
        return self.mapa[car]

In [86]:
class MaquinaEnigma:
    '''Clase para modelar máquina enigma  Wehrmacht con posibilidad de simular una Kriegsmarine'''
    
    '''rotores validos para la Wehrmacht'''
    #rotores_wehrmacht = ["I", "II", "III", "IV", "V"]
    
    '''reflectores validos para ambas máquinas'''
    #reflectores_defecto = ["B", "C"]
    
    '''rotores validos para la Kriegsmarine M3 y M4'''
    #rotores_kriegsmarine = ["I", "II", "III", "IV", "V", "VI", "VII", "VIII"]
    
    '''rotores validos para la Kriegsmarine M4, cuarto rotor'''
    #rotores_kriegsmarine_especiales = ["I", "II", "III", "IV", "V", "VI", "VII", "VIII", "beta", "gamma"]
    
    '''reflectores validos para Kriegsmarine M4'''
    #reflectores_especiales = ["B_thin", "C_thin"]

    def __init__(self, rotores, reflector, ringstellung, grundstellung, clavijas):
        '''Constructor de la clase con el modelo de la maquina, configuracion inicial de los rotores y reflector'''
        
        if len(rotores) != 4 or len(rotores) != len(ringstellung) or len(rotores) != len(grundstellung):
                sys.exit("Error, la máquina Kriegsmarine M4 no puede tener un número de rotores distinto de 4 o las especiifcaciones de ringstellung y grundstellung no se corresponden con los rotores ")
        else:
            rotores_especiales = False
            '''for i in range(0, 3):
                if not (any(rotor == rotores[i] for rotor in MaquinaEnigma.rotores_kriegsmarine)):
                    sys.exit("Error, la máquina Kriegsmarine M4 no puede contener contener alguno de los rotores especificados en la posición dada")
            if rotores[3] == "beta" or rotores[3] == "gamma":
                rotores_especiales = True

            if rotores_especiales:
                if not (any(reflec == reflector for reflec in MaquinaEnigma.reflectores_especiales)):
                    sys.exit("Error, la máquina Kriegsmarine M4 no puede contener ese reflector si se usa el cuarto rotor especial")
            else:
                if not (any(reflec == reflector for reflec in MaquinaEnigma.reflectores_defecto)):
                    sys.exit("Error, la máquina Kriegsmarine M4 no puede contener ese reflector sin usar un cuarto rotor especial")
            '''
        
        
        self.rotores = []
        for rotor in rotores:
            self.rotores.append(Rotor(rotor['notch'], rotor['ringstellung'], rotor['wiring']))
        
        self.ringstellung = ringstellung
        
        for i in range(0, len(ringstellung)):
            self.rotores[i].aplicar_ringstellung(self.ringstellung[i])
            
        self.grundstellung = grundstellung
        
        for i in range(0, len(grundstellung)):
            self.rotores[i].aplicar_grundstellung(self.grundstellung[i])
            
        self.clavijas = clavijas
        
        self.plugboard = Plugboard(self.clavijas)
        
        self.reflector = reflector
    
    
    def cifrar_caracter(self, car):
        car = self.plugboard.sustituir_letra(car)
        
        # Comprobar si los rotores necesitan girar
        girar = [True, False, False, False]
        
        for i in range(0, len(girar)-1):
            if self.rotores[i].comprobar_vuelta_completa():
                girar[i+1] = True
                
        
                
        for i in range(0, len(girar)-1):
            if girar[i]:
                self.rotores[i].rotar()
                
        if girar[2]:
            self.rotores[1].rotar()
        
        for rotor in self.rotores:
            car = rotor.cifrar_caracter(car)
            
        car = self.reflector.reflejar(car)
        
        
        for rotor in reversed(self.rotores):
            car = rotor.inv_cifrar_caracter(car)
            
        car = self.plugboard.sustituir_letra(car)
        
        return car
    
    def cifrar(self, texto):
        texto_cifrado = ""
        for car in texto:
            if (car != " "):
                texto_cifrado = texto_cifrado + self.cifrar_caracter(car)
            
        return texto_cifrado
    
    
        

In [87]:
maquina = MaquinaEnigma([ROTOR_VIII, ROTOR_VI, ROTOR_V, ROTOR_BETA], REFLECTOR_C_THIN, "LEPE", "MEAN", "AE BF")

In [88]:
maquina = MaquinaEnigma([ROTOR_VIII, ROTOR_VI, ROTOR_V, ROTOR_BETA], REFLECTOR_C_THIN, "LEPE", "MEAN", "AE BF CM DQ HU JN LX PR SZ VW")


futuro_grundstellungs = "QEOB"
grundstellungs = maquina.cifrar(futuro_grundstellungs)
print(futuro_grundstellungs)
print(grundstellungs)

# Configuramos la nueva Enigma con el grundstellung que hemos obtenido
enigma = MaquinaEnigma([ROTOR_VIII, ROTOR_VI, ROTOR_V, ROTOR_BETA], REFLECTOR_C_THIN, "LEPE", grundstellungs[::-1], "AE BF CM DQ HU JN LX PR SZ VW")


mensaje = "DUHF TETO LANO TCTO UARB BFPM HPHG CZXT DYGA HGUF XGEW KBLK GJWL QXXT GPJJ AVTO CKZF SLPP QIHZ FXOE BWII EKFZ LCLO AQJU LJOY HSSM BBGW HZAN VOII PYRB RTDJ QDJJ OQKC XWDN BBTY VXLY TAPG VEAT XSON PNYN QFUD BBHH VWEP YEYD OHNL XKZD NWRH DUWU JUMW WVII WZXI VIUQ DRHY MNCY EFUA PNHO TKHK GDNP SAKN UAGH JZSM JBMH VTRE QEDG XHLZ WIFU SKDQ VELN MIMI THBH DBWV HDFY HJOQ IHOR TDJD BWXE MEAY XGYQ XOHF DMYU XXNO JAZR SGHP LWML RECW WUTL RTTV LBHY OORG LGOW UXNX HMHY FAAC QEKT HSJW DUHF TETO"
# Quitar 2 primeros y 2 últimos grupos, que sirven para advertir fallos en la transmisión        
mensaje = mensaje[10:len(mensaje)-10]

mensaje_cifrado = enigma.cifrar(mensaje)

print(f"Mensaje original: {mensaje}")
print(f"Mensaje cifrado: {mensaje_cifrado}")

QEOB
CDSZ
Mensaje original: LANO TCTO UARB BFPM HPHG CZXT DYGA HGUF XGEW KBLK GJWL QXXT GPJJ AVTO CKZF SLPP QIHZ FXOE BWII EKFZ LCLO AQJU LJOY HSSM BBGW HZAN VOII PYRB RTDJ QDJJ OQKC XWDN BBTY VXLY TAPG VEAT XSON PNYN QFUD BBHH VWEP YEYD OHNL XKZD NWRH DUWU JUMW WVII WZXI VIUQ DRHY MNCY EFUA PNHO TKHK GDNP SAKN UAGH JZSM JBMH VTRE QEDG XHLZ WIFU SKDQ VELN MIMI THBH DBWV HDFY HJOQ IHOR TDJD BWXE MEAY XGYQ XOHF DMYU XXNO JAZR SGHP LWML RECW WUTL RTTV LBHY OORG LGOW UXNX HMHY FAAC QEKT HSJW
Mensaje cifrado: KRKRALLEXXFOLGENDESISTSOFORTBEKANNTZUGEBENXXICHHABEFOLGELNBEBEFEHLERHALTENXXJANSTERLEDESBISHERIGXNREICHSMARSCHALLSJGOERINGJSETZTDERFUEHRERSIEYHVRRGRZSSADMIRALYALSSEINENNACHFOLGEREINXSCHRIFTLSCHEVOLLMACHTUNTERWEGSXABSOFORTSOLLENSIESAEMTLICHEMASSNAHMENVERFUEGENYDIESICHAUSDERGEGENWAERTIGENLAGEERGEBENXGEZXREICHSLEITEIKKTULPEKKJBORMANNJXXOBXDXMMMDURNHFKSTXKOMXADMXUUUBOOIEXKP
